# **PolypGen Challenge**

* Eleonora Catania Cecere
* Elisa Cevoli
* Tatiana Camila Cordoba Acosta
* Claudia Ottomano



In [ ]:
# prompt: drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install -q torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install -q mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html

Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Successfully uninstalled torch-2.5.1+cu124
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 621.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.1/94.1 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q mmsegmentation==1.2.2
!pip install -q mmengine==0.10.3
!pip install -q ftfy==6.2.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.1 MB/s eta 0:00:00


# Libraries

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

2.1.0+cu121 True
1.2.2


In [ ]:
# Librerie di base
import os
import mmcv
import mmseg
import cv2
import skimage
import numpy as np
import panda as pd
from PIL import Image
from scipy import ndimage
from scipy.ndimage import convolve, binary_opening, binary_fill_holes

# Librerie per il modello
from mmengine.config import Config
from mmengine.logging import print_log
from mmengine.runner import Runner
from mmseg.registry import DATASETS,RUNNERS
from mmseg.apis import init_model
from mmseg.datasets import BaseSegDataset

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# DATASET

In [ ]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################

# Classes and palette for VERSE dataset
classes = ('background', 'object')
palettePolypGen = [
    (0, 0, 0),       # background - black
    (255, 255, 255)  # object - white
]

@DATASETS.register_module()
class PolypGen(BaseSegDataset):
    METAINFO = dict(classes=classes, palette=palettePolypGen)

    def __init__(self, **kwargs):
        super().__init__(img_suffix='.jpg',
                         seg_map_suffix='_mask.png',
                         reduce_zero_label=False,
                         **kwargs)

PATHs

In [ ]:
group_name = 'Pepine'

In [ ]:
working_folder = f'/content/drive/MyDrive/PolypGen/{group_name}'

checkpoint_path = f'{working_folder}/checkpoint.pth'
config_path = f'{working_folder}/config.py'


'''CARTELLA RISULTATI'''

results_folder  = f'{working_folder}/results_train'

if not os.path.exists(results_folder):
    os.makedirs(results_folder)

data_root = os.path.join('/content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/','Divisione_dataset5')

# Data images
train_img = os.path.join(data_root,'Training','Immagini')
val_img = os.path.join(data_root,'Validation','Immagini')
test_img = os.path.join(data_root,'Test','Immagini')

# Data masks
train_mask = os.path.join(data_root,'Training','Maschere')
val_mask = os.path.join(data_root,'Validation','Maschere')
test_mask  = os.path.join(data_root,'Test','Maschere')

# *Training, Validation e Test*

La **divisione del dataset** è stata effettuata garantendo una equa distribuzione delle immagini rispetto a:

*   Risoluzione delle immagini - *Resolution*
*   Classe (positive/negative) - *Class*
*   Dimensione dei polipi (NA/small/medium/large) - *Polyp_size*
*   Numerosità dei polipi nel frame - *Polyp_number*

Il dataframe qui riportato riassume l'analisi condotta.



In [ ]:
# DATAFRAME

masks_df = pd.read_csv('/content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/Pre-processing/Sistemazione_dataset/masks_df.csv')
masks_df

,Folder,Filename,Sequence,Resolution,Class,Polyp_size,Polyp_number
0,masks_C4,11_endocv2021_positive_1228_mask.jpg,C4,"(1080, 1920)",negative,NA,0
1,masks_C4,10_endocv2021_positive_1087_mask.jpg,C4,"(1080, 1920)",positive,medium,1
2,masks_C4,11_endocv2021_positive_61_mask.jpg,C4,"(1080, 1920)",negative,NA,0
3,masks_C4,11_endocv2021_positive_146_mask.jpg,C4,"(1080, 1920)",negative,NA,0
4,masks_C4,11_endocv2021_positive_625_mask.jpg,C4,"(1080, 1920)",positive,large,1
...,...,...,...,...,...,...,...
3757,masks_seq12,EndoCV2021_seq12_564_mask.jpg,seq12,"(1024, 1280, 3)",positive,small,1
3758,masks_seq12,EndoCV2021_seq12_176_mask.jpg,seq12,"(1024, 1280, 3)",positive,medium,1
3759,masks_seq12,EndoCV2021_seq12_996_mask.jpg,seq12,"(1024, 1280, 3)",positive,medium,1
3760,masks_seq12,EndoCV2021_seq12_324_mask.jpg,seq12,"(1024, 1280, 3)",positive,medium,1


L'aggiunta delle sequenze negative è stata effettuata in base al livello di sfocatura calcolato utilizzando la varianza del filtro Laplaciano, come mostrato nella funzione *is_image_blurry*

In [ ]:
def is_image_blurry(image_path, threshold=80.0):

    image = cv2.imread(image_path)

    # Conversione in scala di grigi
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calcolo della varianza del Laplaciano dell'immagine
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()

    # Verifica se la varianza è inferiore alla soglia
    return fm < threshold, fm


# Preprocessing TRAINING



In [ ]:
''' CROPPING '''

def crop_image_train(img_path, mask_path, threshold_L=18, threshold_space=87, fill=False):

    # 1. Lettura dell'immagine e delle informazioni di base
    image = cv2.imread(img_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    file_name = os.path.basename(img_path)

    # 2. Applicazione di un filtro gaussiano per ridurre il rumore
    image_gaus = cv2.GaussianBlur(image, (7, 7), sigmaX=5)

    # 3. Determinazione dello spazio colore (LAB o HSV) in base alla luminosità media dell'immagine
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean_value = np.mean(gray_img)

    if (mean_value >= threshold_space):
        image_lab = cv2.cvtColor(image_gaus, cv2.COLOR_BGR2LAB)
        image_space = image_lab
        num_iteration_dilation = 2
        num_iteration_erosion = 0
        val_sharp = 9
    else:
        image_hsv = cv2.cvtColor(image_gaus, cv2.COLOR_BGR2HSV)
        image_space = image_hsv
        num_iteration_dilation = 10
        num_iteration_erosion = 6
        val_sharp = 100
        fill = True

    # 4. Applicazione del filtro di sharpening per evidenziare i contorni
    kernel_sharpening =  np.array([[-1, -1, -1], [-1, val_sharp, -1], [-1, -1, -1]]) #15
    sharpened_image = cv2.filter2D(image_space, -1, kernel_sharpening)

    # 5. Creazione di una maschera binaria sulla base del primo canale dell'immagine elaborata
    maschera_binaria = (sharpened_image[...,0] > threshold_L).astype(np.uint8)

    # 6. Applicazione di operazioni morfologiche sulla maschera
    kernel_open = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    maschera_opened = cv2.morphologyEx(maschera_binaria, cv2.MORPH_OPEN, kernel_open, iterations=2)
    maschera_opened = cv2.dilate(maschera_opened, (np.ones((5, 5), np.uint8)), iterations=num_iteration_dilation)

    # Riempimento dei buchi nelle maschere, solo nello spazio HSV
    if fill:
       maschera_opened = binary_fill_holes(maschera_opened).astype(np.uint8) #per sequenza 12

    # Erosione della maschera per affinare la segmentazione
    maschera_opened = cv2.erode(maschera_opened, (np.ones((5, 5), np.uint8)), iterations=num_iteration_erosion)

    # 7. Analisi della metà superiore dell'immagine per determinare il bounding box orizzontale
    height_half = int(image.shape[0] * 3 / 5) # Considera il 60% superiore dell'immagine
    maschera_upper_half = maschera_opened[:height_half, :]
    contours_upper_half, _ = cv2.findContours(maschera_upper_half, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Filtraggio dei contorni validi
    valid_contours_upper_half = [contour for contour in contours_upper_half if cv2.contourArea(contour) > 0]

    max_area_upper_half = 0
    largest_contour_upper_half = None

    # Identificazione del contorno più grande nella metà superiore
    for contour in valid_contours_upper_half:
        area = cv2.contourArea(contour)
        if area > max_area_upper_half:
            max_area_upper_half = area
            largest_contour_upper_half = contour

    # Rettangolo di delimitazione del contorno più grande nella porzione superiore
    if largest_contour_upper_half is not None:
        x_upper_half, y_upper_half, w_upper_half, h_upper_half = cv2.boundingRect(largest_contour_upper_half)
    else:
        x_upper_half, y_upper_half, w_upper_half, h_upper_half = 0, 0, image.shape[1], image.shape[0]  # Se non c'è contorno, usa l'intera immagine

    # 8. Analisi dell'intera immagine per determinare il bounding box verticale
    contours_full_image, _ = cv2.findContours(maschera_opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Filtraggio dei contorni validi
    valid_contours_full_image = [contour for contour in contours_full_image if cv2.contourArea(contour) > 0]

    max_area_full_image = 0
    largest_contour_full_image = None

    # Identificazione del contorno più grande nell'intera immagine
    for contour in valid_contours_full_image:
        area = cv2.contourArea(contour)
        if area > max_area_full_image:
            max_area_full_image = area
            largest_contour_full_image = contour

    # Rettangolo di delimitazione del contorno più grande nell'intera immagine
    if largest_contour_full_image is not None:
        x_full_image, y_full_image, w_full_image, h_full_image = cv2.boundingRect(largest_contour_full_image)
    else:
        x_full_image, y_full_image, w_full_image, h_full_image = 0, 0, image.shape[1], image.shape[0]  # Se non c'è contorno, usa l'intera immagine

    # 9. Creazione dell'immagine e maschera ritagliata utilizzando la larghezza calcolata nella metà superiore e l'altezza su tutta l'immagine
    cropped_image = image[y_full_image:y_full_image+h_full_image, x_upper_half:x_upper_half+w_upper_half]
    cropped_mask = mask[y_full_image:y_full_image+h_full_image, x_upper_half:x_upper_half+w_upper_half]

    # 10. Controllo se il ritaglio è valido o se è troppo piccolo
    original_area = image.shape[0] * image.shape[1]
    cropped_area = cropped_image.shape[0] * cropped_image.shape[1]

    if cropped_area < original_area / 2:
        final_image = image  # Mantieni l'immagine originale
        final_mask = mask # Mantieni la maschera originale

    else:
        final_image = cropped_image  # Usa l'immagine ritagliata
        final_mask = cropped_mask # Usa la maschera ritagliata

    # Converti l'immagine nel formato RGB
    final_image = cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB)

    return final_image, final_mask

''' RESIZE '''

def resize_mask(image_array, target_size, order=0, anti_aliasing=False):
    # Binarizzazione della maschera
    image_array[image_array < 128] = 0
    image_array[image_array >= 128] = 1
    # Ridimensionamento della maschera (interpolazione nearest con order=0 e filtro anti_aliasing false)
    resized_img = skimage.transform.resize(image_array, target_size, anti_aliasing=anti_aliasing, order=order)
    return resized_img

def resize_image_train(image_array, target_size, order=1, anti_aliasing=True):
    # Ridimensionamento dell'immagine (interpolazione bilineare con order=1 e filtro anti_aliasing)
    resized_img = skimage.transform.resize(image_array, target_size, anti_aliasing=anti_aliasing, order=order)
    # Conversione dell'immagine nel range [0, 255] e nel tipo uint8
    resized_img = (resized_img * 255).astype(np.uint8)
    return resized_img

''' RILEVAZIONE E RIMOZIONE DI SPECULAR HIGHLIGHTS '''

def reflex_detection(image_array):
    R = image_array[:, :, 0]  # Canale rosso
    G = image_array[:, :, 1]  # Canale verde
    # Identificazione pixel specular highlights
    mask = np.where((R > 220) & (G > 190), 255, 0).astype(np.uint8)
    return mask

def filter_small_and_large_regions(mask_array, max_area=500, large_area_threshold=1000):

    # Etichettatura delle componenti connesse
    labeled_mask, num_labels = ndimage.label(mask_array)

    # Inizializza le maschere per le regioni piccole e grandi
    mask_small = np.zeros_like(mask_array)
    mask_large = np.zeros_like(mask_array)

    # Iterazione su tutte le regioni identificate
    for label in range(1, num_labels + 1):
        region = (labeled_mask == label)
        area = np.sum(region)

        # Se l'area è piccola, maschera delle aree piccole
        if area < max_area:
            mask_small[region] = 255
        # Se l'area è grande ma inferiore ad una soglia, maschera delle aree grandi
        elif area >= max_area and area < large_area_threshold:
            mask_large[region] = 255

    return mask_small, mask_large

def dilate_mask(mask_array, kernel_size=5):
    # Kernel rettangolare per la dilatazione
    structure = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    dilated_mask = ndimage.binary_dilation(mask_array, structure=structure).astype(np.uint8) * 255
    return dilated_mask

def apply_opening(mask_array, kernel_size=5):
    # Kernel per l'operazione morfologica
    structure = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    processed_mask = binary_opening(mask_array, structure=structure).astype(np.uint8) * 255
    return Image.fromarray(processed_mask)

def calculate_texture_features(image, mask):
    # Calcolo delle caratteristiche della texture per i pixel riflettenti

    # 1. Conversione dell'immagine in scala di grigi
    gray_image = np.array(image.convert("L"))

    # 2. Definizione dei filtri di Sobel per il calcolo dei gradienti in 4 direzioni principali
    G_0 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]]) # Direzione 0° (verticale)
    G_90 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]) # Direzione 90° (orizzontale)
    G_45 = np.array([[-2, -1, 0], [-1, 0, 1], [0, 1, 2]]) # Direzione 45° (diagonale: topLeft - downRight)
    G_135 = np.array([[0, 1, 2], [-1, 0, 1], [-2, -1, 0]]) # Direzione 135° (diagonale: topRight - downLeft)

    # 3. Applicazione dei filtri di Sobel per calcolare i gradienti in ogni direzione
    gradients = {
        0: convolve(gray_image, G_0, mode='nearest'),
        45: convolve(gray_image, G_45, mode='nearest'),
        90: convolve(gray_image, G_90, mode='nearest'),
        135: convolve(gray_image, G_135, mode='nearest'),
    }

    # 4. Calcolo del gradiente massimo tra tutte le direzioni
    max_gradient = np.maximum.reduce([gradients[0], gradients[45], gradients[90], gradients[135]])

    # 5. Creazione della matrice K per memorizzare la direzione del gradiente massimo
    K = np.zeros_like(max_gradient)

    # 6. Determinazione della direzione dominante per ogni pixel
    for angle, grad in gradients.items():
        K[max_gradient == grad] = angle

    # 7. Applicazione della maschera: i pixel non riflettenti sono impostati a 0
    K[mask == 0] = 0
    return K

def reflected_light_compensation(image_array, mask_array):
    # Compensazione delle aree riflettenti dell'immagine sostituendo i pixel riflettenti con pixel vicini non riflettenti

    # 1. Calcolo della direzione del gradiente per ciascun pixel riflettente
    K = calculate_texture_features(Image.fromarray(image_array), Image.fromarray(mask_array))

    compensated_image = image_array.copy()

    # 2. Identificazione dei pixel riflettenti (255 nella maschera)
    rows, cols = np.where(mask_array == 255)

    # 3. Per ogni pixel riflettente, cerca il primo pixel non riflettente nella direzione del gradiente K
    for x, y in zip(rows, cols):
        direction = K[x, y] # Ottieni la direzione del gradiente

        # 4. Determina la direzione di ricerca (dx, dy)
        dx, dy = 0, 0
        if direction == 0:  # Direzione orizzontale
            dx, dy = 0, 1
        elif direction == 45:  # Direzione 45 gradi
            dx, dy = -1, 1
        elif direction == 90:  # Direzione verticale
            dx, dy = -1, 0
        elif direction == 135:  # Direzione 135 gradi
            dx, dy = -1, -1

        # 5. Cerca il primo pixel non riflettente lungo la direzione calcolata
        nx, ny = x + dx, y + dy
                             # heigh                           # width
        while 0 <= nx < image_array.shape[0] and 0 <= ny < image_array.shape[1]:
            pixel_value = compensated_image[nx, ny] # Ottieni il valore RGB del pixel

            # Determina se il pixel è nero (soglia RGB < 50 per ogni canale)
            is_black = all(channel < 50 for channel in pixel_value)

            # Se il pixel non è riflettente e non è nero, usalo per sostituire il riflesso
            if mask_array[nx, ny] == 0 and not is_black:
                compensated_image[x, y] = compensated_image[nx, ny] # Sostituisci il pixel riflettente
                break

            # Continua nella stessa direzione
            nx, ny = nx + dx, ny + dy

    return compensated_image


def rgb_mask_inpainting(image_array, mask_array, patch_size=75):

    compensated_image = image_array.copy()

    # 1. Determina la metà del patch per definire l'area di ricerca
    half_patch = patch_size // 2  # Metà dimensione del patch

    # 2. Trova i pixel da ricostruire in base alla maschera (su tutti i canali)
    rows, cols, channels = np.where(mask_array > 0)

    # 3. Itera su tutti i pixel da ricostruire
    for x, y, channel in zip(rows, cols, channels):
        # 4. Definisce i limiti dell'intorno (evita di uscire dai bordi dell'immagine)
        x_min = max(x - half_patch, 0)
        x_max = min(x + half_patch + 1, image_array.shape[0])
        y_min = max(y - half_patch, 0)
        y_max = min(y + half_patch + 1, image_array.shape[1])

        # 5. Estrai la regione di interesse (patch) dall'immagine e dalla maschera
        patch = compensated_image[x_min:x_max, y_min:y_max]
        patch_mask = mask_array[x_min:x_max, y_min:y_max, channel]

        # 6. Filtra i pixel validi:
        # - Devono appartenere alla patch
        # - Non devono essere riflettenti (patch_mask == 0)
        # - Non devono essere troppo scuri (tutti i valori RGB > 25)
        valid_pixels = patch[(patch_mask == 0) & (np.all(patch > 25, axis=-1))]

        if len(valid_pixels) > 0:
            # 7. Se esistono pixel validi, sostituisce il valore riflettente con la media della regione
            compensated_image[x, y, channel] = np.mean(valid_pixels[:, channel], axis=0).astype(np.uint8)
        else:
            # 8. Se non ci sono pixel validi, mantiene il valore originale
            compensated_image[x, y, channel] = image_array[x, y, channel]

    return compensated_image


def remove_specular_highlights(original_image_array):

    # Step 1: Rilevazione degli specular highlights
    mask = reflex_detection(original_image_array)

    # Step 2: Filtraggio delle regioni piccole e grandi
    mask_small, mask_large = filter_small_and_large_regions(mask)

    # Step 3: Dilatazione della maschera small e apertura della maschera large
    refined_mask_small = dilate_mask(mask_small)
    refined_mask_large = apply_opening(mask_large)

    # Combinazione delle maschere
    refined_mask = refined_mask_small | refined_mask_large

    # Step 4: Ricostruzione dell'immagine sostituendo i riflessi con pixel adiacenti
    reconstructed_img = reflected_light_compensation(original_image_array, refined_mask)

    # Step 5: Applicazione di un filtro bilaterale SOLO alle aree della maschera
    filtered_masked_areas = cv2.bilateralFilter(
        cv2.bitwise_and(reconstructed_img, reconstructed_img, mask=dilate_mask(refined_mask)),
        d=9, sigmaColor=75, sigmaSpace=75
    )

    # Sostituisce le zone filtrate con interpolazione locale (inpainting basato su patch)
    image_with_selective_filtering = rgb_mask_inpainting(reconstructed_img, np.array(filtered_masked_areas), patch_size=15)

    # Step 6: Applicazione di un filtro mediano all'intera immagine per ridurre il rumore
    median_image_with_selective_filtering = cv2.medianBlur(image_with_selective_filtering, 5)

    # Step 7: Sharpening per ripristinare i dettagli dell'immagine dopo la filtratura
    kernel_sharpening = np.array([[-0.5, -0.5, -0.5], [-0.5, 5, -0.5], [-0.5, -0.5, -0.5]])
    sharpened_image = cv2.filter2D(median_image_with_selective_filtering, -1, kernel_sharpening)

    return sharpened_image

''' GAMMA CORRECTION '''

def gamma_correction(image_array, gamma = 1.2):
    gamma_corrected = np.zeros_like(image_array)
    look_up_table = np.array([((i / 255.0) ** (1.0 / gamma)) * 255 for i in range(256)], dtype=np.uint8)
    return cv2.LUT(image_array, look_up_table)

In [ ]:
def preprocess_images(input_images, output_images, input_mask, output_mask, target_size=(256, 256)):
    os.makedirs(output_images, exist_ok=True)
    os.makedirs(output_mask, exist_ok=True)

    for filename in os.listdir(input_images):
        if filename.endswith('.jpg'):
            # Path de imagen y máscara
            image_path = os.path.join(input_images, filename)
            mask_path = os.path.join(input_mask, filename.replace('.jpg', '_mask.jpg'))

            ''' CROPPING '''
            img_cropped, mask_cropped = crop_image_train(image_path, mask_path)

            ''' RESIZE '''
            img_resize = resize_image_train(img_cropped, target_size)
            mask_resize = resize_mask(mask_cropped, target_size)

            ''' SPECULAR HIGHLIGHTS REMOVE '''
            img_specH = remove_specular_highlights(img_resize)

            ''' GAMMA CORRECTION '''
            img_gamma_corrected = gamma_correction(img_specH)

            ''' SALVATAGGIO IMMAGINI '''
            output_image = Image.fromarray(img_gamma_corrected.astype(np.uint8))
            output_image_path = os.path.join(output_images, filename)
            output_image.save(output_image_path, format='JPEG')

            ''' SALVATAGGIO MASCHERE '''
            output_mask_image = Image.fromarray(mask_resize.astype(np.uint8))
            output_mask_path = os.path.join(output_mask, filename.replace('.jpg', '_mask.jpg'))
            output_mask_image.save(output_mask_path, format='PNG')


In [ ]:
#output images
train_img_prep = os.path.join(data_root,'Training','img_crop_res_specH_gamma')
val_img_prep = os.path.join(data_root,'Validation','img_crop_res_specH_gamma')
test_img_prep = os.path.join(data_root,'Test','img_crop_res_specH_gamma')

# Data masks
train_mask_prep = os.path.join(data_root,'Training','mask_crop_res')
val_mask_prep = os.path.join(data_root,'Validation','mask_crop_res')
test_mask_prep  = os.path.join(data_root,'Test','mask_crop_res')

In [ ]:
#Pre-processing immagini e maschere Training
preprocess_images(train_img, train_img_prep, train_mask, train_mask_prep)

#Pre-processing immagini e maschere Validation
preprocess_images(val_img, val_img_prep, val_mask, val_mask_prep)

#Pre-processing immagini e maschere Test
preprocess_images(test_img, test_img_prep, test_mask, test_mask_prep)

# RETE

**Configurazione di DEEPLABV3+**

In [ ]:
# Config loading
cfg = Config.fromfile(config_path)
cfg.device='cuda'

**Modifica della configurazione della rete DEEPLABV3+ per gestire il dataset PolypGen**



In [ ]:
# Definizione cartella contenente il dataset e classi
n_classes=len(classes) # Numero di Classi  (polipo e sfondo)

norm_cfg = dict(type='BN')

# Stessa configurazione di normalizzazione per ogni passaggio.
cfg.model.auxiliary_head.norm_cfg = norm_cfg
cfg.model.backbone.norm_cfg = norm_cfg
cfg.model.decode_head.norm_cfg = norm_cfg
cfg.norm_cfg = norm_cfg

# Numero di classi da segmentare.
cfg.model.auxiliary_head.num_classes = n_classes
cfg.model.decode_head.num_classes = n_classes

# Dimensioni dell'immagine (256x256).
image_size = (256,256)
cfg.crop_size = image_size
cfg.data_preprocessor.size = image_size
cfg.model.data_preprocessor.size = image_size

# Mantiene i valori originali senza applicare la normalizzazione
cfg.data_preprocessor.mean = [0, 0, 0]
cfg.data_preprocessor.std = [1, 1, 1]
cfg.model.data_preprocessor.mean = [0, 0, 0]
cfg.model.data_preprocessor.std = [1, 1, 1]

# Fattori di scala per l'augmentazione delle immagini
cfg.img_ratios = [0.8,0.95,1.0,1.05,1.15]

'''TRAIN PIPELINE'''

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label=False),
    dict(
        type='RandomResize',
        scale=(256,256), # Scala 256x256
        ratio_range=(0.95, 1.05), # Variazione 95%-105%
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(256,256), cat_max_ratio=0.95), # Ritaglio casuale
    dict(type='RandomFlip', prob=0.5), # Flip casuale (50%)
    dict(type='PhotoMetricDistortion'), # Distorsione fotometrica
    dict(type='PackSegInputs')
]

'''TEST PIPELINE'''

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256,256), keep_ratio=True),
    dict(type='LoadAnnotations', reduce_zero_label=False),
    dict(type='PackSegInputs')
]

In [ ]:
''' OPTIMIZER SGD '''

cfg.optim_wrapper.optimizer.lr = 0.001
cfg.optimizer.lr = 0.001

''' LOSS FUNCTION '''
cfg.model.decode_head.loss_decode = [

     # Peso della classe per il polipo
    dict(
        type='CrossEntropyLoss',
        loss_weight=0.9,   # Maggiore peso per dare priorità al polipo
        use_sigmoid=False
    ),

    # Peso della classe per lo sfondo
    dict(
        type='CrossEntropyLoss',
        loss_weight=0.1, #0.1,  # Minore peso per lo sfondo
        use_sigmoid=False
    )
]

''' CHECKPOINT '''

cfg.default_hooks.checkpoint=dict(
        by_epoch=False,
        interval=4000,
        type='CheckpointHook',
        save_best='mDice', #Salva il modello con il miglior punteggio mDice.
        rule='greater',
        max_keep_ckpts = 1 # Solo salva un checkpoint
    )

''' EARLY STOPPING '''

cfg.default_hooks.early_stopping = dict(
    monitor='mIoU', # metrica da monitorare
    rule='greater',
    patience=20, # numero massimo di iterazioni senza miglioramento
    min_delta=0.01,  # variazione minima
    type='EarlyStoppingHook')


Dataloader

In [ ]:
''' IMMAGINI TRAIN, VAL e TEST '''

#path immagine train_dataloader
path_train_dataloader = train_img_prep

#path immagine val_dataloader
path_val_dataloader = val_img_prep

#path immagine test_dataloader
path_test_dataloader = test_img_prep

''' MASCHERE TRAIN, VAL e TEST '''

#path maschere train_dataloader
path_train_mask = train_mask_prep

#path maschere val_dataloader
path_val_mask = val_mask_prep

#path maschere test_dataloader
path_test_mask = test_mask_prep

In [ ]:
#  Imposta il nome del dataset e la directory root
cfg.dataset_type = 'PolypGen' # dataset name
cfg.data_root = data_root
batch_size = 6

'''TRAIN DATALOADER'''

cfg.train_dataloader.batch_size = batch_size
cfg.train_dataloader.dataset.type = 'PolypGen'
cfg.train_dataloader.dataset.data_root = data_root
cfg.train_dataloader.dataset.data_prefix.img_path = path_train_dataloader
cfg.train_dataloader.dataset.data_prefix.seg_map_path = path_train_mask
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline

'''VALIDATION DATALOADER'''

cfg.val_dataloader.batch_size = batch_size
cfg.val_dataloader.dataset.type = 'PolypGen'
cfg.val_dataloader.dataset.data_root = data_root
cfg.val_dataloader.dataset.data_prefix.img_path = path_val_dataloader
cfg.val_dataloader.dataset.data_prefix.seg_map_path = path_val_mask
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_evaluator.iou_metrics = ['mIoU','mDice']

'''TEST DATALOADER'''

cfg.test_dataloader.batch_size = 1
cfg.test_dataloader.dataset.type = 'PolypGen'
cfg.test_dataloader.dataset.data_root = data_root
cfg.test_dataloader.dataset.data_prefix.img_path = path_test_dataloader
cfg.test_dataloader.dataset.data_prefix.seg_map_path = path_test_mask
cfg.test_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.test_evaluator.iou_metrics = ['mIoU', 'mDice']

In [ ]:
'''PARAMETRI DI TRAINING e VALIDAITION'''

# lunghezza del dataset di training
train_length = len(os.listdir(train_img))
refresh_val = 1
# Flag per caricare un modello pre-allenato
flag_pretrained = True

iter_for_epoch = round(train_length / batch_size)
max_iters = 20000  # Numero massimo di iterazioni
interval_val = round(refresh_val * iter_for_epoch)  # Intervallo per la validazione
# Configurazione del training loop
cfg.train_cfg = dict(type='IterBasedTrainLoop', max_iters=max_iters, val_interval=interval_val)

# Configurazione dei loop di validazione e test (default)
cfg.val_cfg = dict(type='ValLoop')
cfg.test_cfg = dict(type='TestLoop')

cfg.workflow = [('train', 1), ('val', 1)]

cfg.load_from = checkpoint_path if flag_pretrained else None

In [ ]:
''' INIZIALIZZAZIONE MODELLO '''

path_weights = 'weights_deeplabV3+'

model = init_model(cfg)

# Configurazione della visualizzazione
cfg.visualizer.save_dir = os.path.join(results_folder,path_weights)
cfg.visualizer.classes = classes
cfg.visualizer.palette = palettePolypGen # Palette dei colori per la visualizzazione
cfg.default_hooks.checkpoint.interval = interval_val

cfg.update(dict(work_dir=os.path.join(results_folder,path_weights)))
cfg.update(dict(launcher='none'))

# Creazione del runner
runner = RUNNERS.build(cfg)

/usr/local/lib/python3.11/dist-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/usr/local/lib/python3.11/dist-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/usr/local/lib/python3.11/dist-packages/mmseg/models/losses/cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


02/02 22:13:38 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1820816569
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=s

/usr/local/lib/python3.11/dist-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmseg.visualization.local_visualizer.SegLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


02/02 22:13:40 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
02/02 22:13:40 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
(LOWEST      ) EarlyStoppingHook                  
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_i

/usr/local/lib/python3.11/dist-packages/mmseg/engine/hooks/visualization_hook.py:60: UserWarning: The draw is False, it means that the hook for visualization will not take effect. The results will NOT be visualized or stored.
  warnings.warn('The draw is False, it means that the '


In [ ]:
# Visualizzazione del config caricato
print(f'Config:\n{cfg.pretty_text}')

Config:
crop_size = (
    256,
    256,
)
data_preprocessor = dict(
    bgr_to_rgb=True,
    mean=[
        0,
        0,
        0,
    ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        256,
        256,
    ),
    std=[
        1,
        1,
        1,
    ],
    type='SegDataPreProcessor')
data_root = '/content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/Divisione_dataset5'
dataset_type = 'PolypGen'
default_hooks = dict(
    checkpoint=dict(
        by_epoch=False,
        interval=448,
        max_keep_ckpts=1,
        rule='greater',
        save_best='mDice',
        type='CheckpointHook'),
    early_stopping=dict(
        min_delta=0.01,
        monitor='mIoU',
        patience=20,
        rule='greater',
        type='EarlyStoppingHook'),
    logger=dict(interval=50, log_metric_by_epoch=False, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=d

In [ ]:
runner.train()

/usr/local/lib/python3.11/dist-packages/mmseg/datasets/transforms/loading.py:83: UserWarning: `reduce_zero_label` will be deprecated, if you would like to ignore the zero label, please set `reduce_zero_label=True` when dataset initialized
  warnings.warn('`reduce_zero_label` will be deprecated, '


02/02 18:43:11 - mmengine - WARNING - The prefix is not set in metric class IoUMetric.


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Loads checkpoint by local backend from path: /content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/Confronto_reti/Training_resnet50_dataset5/best_ckpt8261/best_mDice_iter_3584.pth
02/02 18:43:28 - mmengine - INFO - Load checkpoint from /content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/Confronto_reti/Training_resnet50_dataset5/best_ckpt8261/best_mDice_iter_3584.pth
02/02 18:43:28 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
02/02 18:43:28 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
02/02 18:43:28 - mmengine - INFO - Checkpoints will be saved to /content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/Confronto_reti/Training_resnet50_dataset5/Pepine/results/weights_deeplabV3+.
02/02 18:45:34 - mmengine - INFO - Iter(train) [   50/20000]  lr: 9.9901e-04  eta: 13:59:52  time: 1.4603  data_time: 0.9960  memory: 119

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


02/02 18:54:16 - mmengine - INFO - Iter(val) [50/90]    eta: 0:00:46  time: 1.2085  data_time: 1.0443  memory: 3165  
02/02 18:54:59 - mmengine - INFO - per class results:
02/02 18:54:59 - mmengine - INFO - 
+------------+-------+-------+-------+
|   Class    |  IoU  |  Acc  |  Dice |
+------------+-------+-------+-------+
| background | 96.84 | 99.01 |  98.4 |
|   object   | 68.75 | 76.06 | 81.48 |
+------------+-------+-------+-------+
02/02 18:54:59 - mmengine - INFO - Iter(val) [90/90]    aAcc: 97.0500  mIoU: 82.7900  mAcc: 87.5300  mDice: 89.9400  data_time: 0.9631  time: 1.1213
02/02 18:55:00 - mmengine - INFO - The best checkpoint with 89.9400 mDice at 448 iter is saved to best_mDice_iter_448.pth.
02/02 18:55:06 - mmengine - INFO - Iter(train) [  450/20000]  lr: 9.9090e-04  eta: 7:11:04  time: 1.5613  data_time: 1.0862  memory: 3165  loss: 0.0165  decode.loss_ce: 0.0091  decode.acc_seg: 99.4293  aux.loss_ce: 0.0074  aux.acc_seg: 98.8717
02/02 18:55:30 - mmengine - INFO - Iter(tr

EncoderDecoder(
  (data_preprocessor): SegDataPreProcessor()
  (backbone): ResNetV1c(
    (stem): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-0

In [ ]:
runner.test()

02/02 22:14:12 - mmengine - WARNING - The prefix is not set in metric class IoUMetric.


/usr/local/lib/python3.11/dist-packages/mmseg/datasets/transforms/loading.py:83: UserWarning: `reduce_zero_label` will be deprecated, if you would like to ignore the zero label, please set `reduce_zero_label=True` when dataset initialized
  warnings.warn('`reduce_zero_label` will be deprecated, '


Loads checkpoint by local backend from path: /content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/Confronto_reti/Training_resnet50_dataset5/best_ckpt8261/best_mDice_iter_3584.pth
02/02 22:14:14 - mmengine - INFO - Load checkpoint from /content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/Confronto_reti/Training_resnet50_dataset5/best_ckpt8261/best_mDice_iter_3584.pth


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


02/02 22:14:18 - mmengine - INFO - Iter(test) [ 50/532]    eta: 0:00:35  time: 0.0327  data_time: 0.0053  memory: 677  
02/02 22:14:20 - mmengine - INFO - Iter(test) [100/532]    eta: 0:00:23  time: 0.0380  data_time: 0.0049  memory: 666  
02/02 22:14:21 - mmengine - INFO - Iter(test) [150/532]    eta: 0:00:18  time: 0.0380  data_time: 0.0035  memory: 666  
02/02 22:14:23 - mmengine - INFO - Iter(test) [200/532]    eta: 0:00:14  time: 0.0306  data_time: 0.0017  memory: 666  
02/02 22:14:25 - mmengine - INFO - Iter(test) [250/532]    eta: 0:00:11  time: 0.0309  data_time: 0.0016  memory: 666  
02/02 22:14:26 - mmengine - INFO - Iter(test) [300/532]    eta: 0:00:09  time: 0.0307  data_time: 0.0016  memory: 666  
02/02 22:14:28 - mmengine - INFO - Iter(test) [350/532]    eta: 0:00:06  time: 0.0319  data_time: 0.0016  memory: 666  
02/02 22:14:29 - mmengine - INFO - Iter(test) [400/532]    eta: 0:00:04  time: 0.0298  data_time: 0.0015  memory: 682  
02/02 22:14:31 - mmengine - INFO - Iter(

{'aAcc': 96.77, 'mIoU': 75.3, 'mAcc': 83.2, 'mDice': 84.19}